# 3 - Normalized Score and Color Calculator
-----
    This notebook was used to pare down a large amount of tabular census data to extract and calculate scores on the 81 zipcodes targeted for this project. By collecting some data preprocessed in either steps 1 or 2, we're able to marry that to data compiled from the US Census. This notebook outputs a primary data table for group analysis, as well as calculates normalized scores for specific values, then converts those values into color values based on the Matplotlib RdYlGn (Red Yellow Green) colormap. These color values will be used to populate the fill color values for the choropleth maps that will be generated in step 4.

In [1]:
import pandas as pd
from matplotlib.cm import RdYlGn
from matplotlib.colors import to_hex
import ast
import json

In [2]:
# Open your CSVs as a dataframes
    
with open('../all_econ_data.csv', newline='\n', encoding='utf-8-sig') as a:
    all_econ_data = pd.read_csv(a)
    
with open('../zcta_map/resources/zip_data.csv', newline='\n', encoding='utf-8-sig') as b:
    zip_data = pd.read_csv(b)
    
with open('../hcprovider_data/doctor_geo.csv', newline='\n', encoding='utf-8-sig') as c:
    doctor_geo = pd.read_csv(c)
    
with open('../hcprovider_data/hospital_geo.csv', newline='\n', encoding='utf-8-sig') as d:
    hospital_geo = pd.read_csv(d)
    
with open('../insurance_data/insurance_data.csv', encoding='utf-8') as e:
    ins_data = pd.read_csv(e)

In [3]:
all_econ_data.head()

,Zip Code,Total Population,Labor Force Participation Rate,Unemployment Rate,Per Capita Income,"Total households - Less than $10,000","Total households - \$10,000-$14,999","Total households - \$15,000-$24,999","Total households - \$25,000-$34,999","Total households - \$35,000-$49,999",...,"Total households - \$75,000-$99,999","Total households - \$100,000-$149,999","Total households - \$150,000-$199,999","Total households - \$200,000 or more",Total Households,Median Income by Household,Total Families,Median Income by Families,Total Nonfamily Households,Median Income by Nonfamily Households
0,20001,41692,73,5,54353.0,1730,885,1302,876,1072,...,1710,3765,2191,3475,18764,100447,6040,113317,12724,93786
1,20002,52867,72,6,49452.0,2501,1163,1559,1504,2142,...,2894,4720,2729,3704,26217,82022,10665,101998,15552,69683
2,20003,26330,71,5,63600.0,686,367,456,352,417,...,1496,2566,1840,2852,12394,118607,5296,167746,7098,94339
3,20004,1610,88,4,117432.0,48,8,9,20,63,...,102,329,166,339,1163,144583,280,248672,883,135189
4,20005,12311,84,2,76658.0,751,212,304,425,446,...,1292,1602,839,1318,8172,94506,1668,142821,6504,83333


In [4]:
zip_data.head()

,zip,city,county,median_household_income,population,feature,area,density,nml_dens,nml_inc,color_key
0,20001,"Washington, DC",District Of Columbia,100447.0,41692,"{'type': 'Feature', 'geometry': {'type': 'Poly...",5.835524,7144.516970,0.759751,0.386274,#0062ce
1,20002,"Washington, DC",District Of Columbia,82022.0,52867,"{'type': 'Feature', 'geometry': {'type': 'Poly...",14.228399,3715.597280,0.964587,0.284281,#0048db
2,20003,"Washington, DC",District Of Columbia,118607.0,26330,"{'type': 'Feature', 'geometry': {'type': 'Poly...",6.447784,4083.573672,0.478169,0.486800,#007cc1
3,20004,"Washington, DC",District Of Columbia,144583.0,1610,"{'type': 'Feature', 'geometry': {'type': 'Mult...",0.904905,1779.192084,0.025057,0.630593,#00a1ae
4,20005,"Washington, DC",District Of Columbia,94506.0,12311,"{'type': 'Feature', 'geometry': {'type': 'Poly...",1.122787,10964.678013,0.221204,0.353387,#005ad2


In [5]:
doctor_geo.head()

,Unnamed: 0,lat,lng,name,vicinity,placeid,zip
0,0,38.917568,-77.020726,Howard University Hospital:pulmonary,"2041 Georgia Avenue Northwest, Washington",ChIJf2TW1OS3t4kRmYwM_D3YU9g,20001
1,1,38.905311,-77.012765,"Shiban Raina, MD","128 M Street Northwest #050, Washington",ChIJqVHvjcHHt4kR9bkSKfeYk6M,20001
2,2,38.905363,-77.018377,Aziz Salim MD,"2175 K Street Northwest, Washington",ChIJO7BERbS3t4kRdCq1JjRVF9c,20001
3,3,38.910311,-77.021587,Abramson Randi C MD,"1525 7th Street Northwest, Washington",ChIJNcEa_fG3t4kROr29e7EFZJw,20001
4,4,38.910311,-77.021587,"Hope Ferdowsian, MD, MPH","1525 7th Street Northwest, Washington",ChIJxfO-cZTJt4kR1eicxoihlSc,20001


In [6]:
ins_data.head()

,GEO.id2,GEO.id2.1,HC03_VC131
0,0,Id2,Percent; HEALTH INSURANCE COVERAGE - Civilian ...
1,1,20001,96.0
2,2,20002,95.4
3,3,20003,97.6
4,4,20004,98.7


In [7]:
dr_grpby = doctor_geo[['placeid','zip']].groupby('zip')
dr_df = pd.DataFrame(dr_grpby.count())

hos_grpby = hospital_geo[['placeid','zip']].groupby('zip')
hos_df = pd.DataFrame(hos_grpby.count())

In [8]:
dr_df.head()

,placeid
zip,
20001,32
20002,58
20003,49
20004,14
20005,20


In [9]:
hos_df.head()

,placeid
zip,
20001,11
20002,8
20003,8
20004,5
20005,6


In [10]:
ins_data1 = ins_data.drop(0).rename(index=str, columns={"GEO.id2.1": "zip", "HC03_VC131": "Percent Insured"}).drop(['GEO.id2'], axis=1)

ins_data1.head()

,zip,Percent Insured
1,20001,96.0
2,20002,95.4
3,20003,97.6
4,20004,98.7
5,20005,95.5


In [11]:
all_econ_data = all_econ_data.rename(index=str, columns={'Zip Code': 'zip'})

# all_econ_data = all_econ_data.set_index('zip')

all_econ_data.head()

,zip,Total Population,Labor Force Participation Rate,Unemployment Rate,Per Capita Income,"Total households - Less than $10,000","Total households - \$10,000-$14,999","Total households - \$15,000-$24,999","Total households - \$25,000-$34,999","Total households - \$35,000-$49,999",...,"Total households - \$75,000-$99,999","Total households - \$100,000-$149,999","Total households - \$150,000-$199,999","Total households - \$200,000 or more",Total Households,Median Income by Household,Total Families,Median Income by Families,Total Nonfamily Households,Median Income by Nonfamily Households
0,20001,41692,73,5,54353.0,1730,885,1302,876,1072,...,1710,3765,2191,3475,18764,100447,6040,113317,12724,93786
1,20002,52867,72,6,49452.0,2501,1163,1559,1504,2142,...,2894,4720,2729,3704,26217,82022,10665,101998,15552,69683
2,20003,26330,71,5,63600.0,686,367,456,352,417,...,1496,2566,1840,2852,12394,118607,5296,167746,7098,94339
3,20004,1610,88,4,117432.0,48,8,9,20,63,...,102,329,166,339,1163,144583,280,248672,883,135189
4,20005,12311,84,2,76658.0,751,212,304,425,446,...,1292,1602,839,1318,8172,94506,1668,142821,6504,83333


In [12]:
econ_select = all_econ_data[['zip', 'Total Population', 'Unemployment Rate', 'Per Capita Income']]

econ_select.head()

,zip,Total Population,Unemployment Rate,Per Capita Income
0,20001,41692,5,54353.0
1,20002,52867,6,49452.0
2,20003,26330,5,63600.0
3,20004,1610,4,117432.0
4,20005,12311,2,76658.0


In [13]:
ins_data2 = ins_data1
ins_data2['Percent Insured'] = ins_data2['Percent Insured'].replace('-',0)
ins_data2 = ins_data2.astype({'zip': 'int64', 'Percent Insured':'float64'})

ins_data2.head()

,zip,Percent Insured
1,20001,96.0
2,20002,95.4
3,20003,97.6
4,20004,98.7
5,20005,95.5


In [14]:
econ_select1 = econ_select.merge(ins_data2, on='zip', how='inner')

econ_spc = econ_select1.set_index('zip')

ziplist = list(zip_data['zip'])

econ_select1 = econ_spc.loc[ziplist]

econ_select1 = econ_select1.reset_index()

In [15]:
econ_select2 = econ_select1.merge(hos_df, on='zip', how='left')
econ_select2 = econ_select2.rename(index=str, columns={'placeid': 'hospital facilities'})
econ_select2 = pd.DataFrame(econ_select2)

econ_select2.head()

,zip,Total Population,Unemployment Rate,Per Capita Income,Percent Insured,hospital facilities
0,20001,41692,5,54353.0,96.0,11.0
1,20002,52867,6,49452.0,95.4,8.0
2,20003,26330,5,63600.0,97.6,8.0
3,20004,1610,4,117432.0,98.7,5.0
4,20005,12311,2,76658.0,95.5,6.0


In [16]:
econ_select3 = econ_select2.merge(dr_df, on='zip', how='left')
econ_select3 = econ_select3.rename(index=str, columns={'placeid': 'doctors'})
econ_select3 = pd.DataFrame(econ_select3)

econ_select3.head()

,zip,Total Population,Unemployment Rate,Per Capita Income,Percent Insured,hospital facilities,doctors
0,20001,41692,5,54353.0,96.0,11.0,32.0
1,20002,52867,6,49452.0,95.4,8.0,58.0
2,20003,26330,5,63600.0,97.6,8.0,49.0
3,20004,1610,4,117432.0,98.7,5.0,14.0
4,20005,12311,2,76658.0,95.5,6.0,20.0


In [17]:
def calculate_color(pop):
    mpl_color = RdYlGn(pop)
    gmaps_color = to_hex(mpl_color, keep_alpha=False)
    return gmaps_color

In [18]:
min_emp = min(econ_select3['Unemployment Rate'])
max_emp = max(econ_select3['Unemployment Rate'])
emp_range = max_emp - min_emp

econ_select3['nml_emp'] = (1 - (econ_select3['Unemployment Rate'] - min_emp)/emp_range)

econ_select3.head()

,zip,Total Population,Unemployment Rate,Per Capita Income,Percent Insured,hospital facilities,doctors,nml_emp
0,20001,41692,5,54353.0,96.0,11.0,32.0,0.782609
1,20002,52867,6,49452.0,95.4,8.0,58.0,0.739130
2,20003,26330,5,63600.0,97.6,8.0,49.0,0.782609
3,20004,1610,4,117432.0,98.7,5.0,14.0,0.826087
4,20005,12311,2,76658.0,95.5,6.0,20.0,0.913043


In [19]:
min_inc = min(econ_select3['Per Capita Income'])
max_inc = max(econ_select3['Per Capita Income'])
inc_range = max_inc - min_inc

econ_select3['nml_inc'] = ((econ_select3['Per Capita Income'] - min_inc)/inc_range)

In [20]:
econ_select3['nml_ins'] = (econ_select3['Percent Insured'] / 100)

econ_select3.head()

,zip,Total Population,Unemployment Rate,Per Capita Income,Percent Insured,hospital facilities,doctors,nml_emp,nml_inc,nml_ins
0,20001,41692,5,54353.0,96.0,11.0,32.0,0.782609,0.383988,0.960
1,20002,52867,6,49452.0,95.4,8.0,58.0,0.739130,0.336126,0.954
2,20003,26330,5,63600.0,97.6,8.0,49.0,0.782609,0.474292,0.976
3,20004,1610,4,117432.0,98.7,5.0,14.0,0.826087,1.000000,0.987
4,20005,12311,2,76658.0,95.5,6.0,20.0,0.913043,0.601813,0.955


In [21]:
colors = []

for pop in econ_select3['nml_inc'].tolist():
    try:
        color = calculate_color(pop)
    except KeyError:
        color = (0, 0, 0, 0.3)
        print('key error')
    colors.append(color)

In [22]:
econ_select3['color_key_inc'] = colors

econ_select3.head()

,zip,Total Population,Unemployment Rate,Per Capita Income,Percent Insured,hospital facilities,doctors,nml_emp,nml_inc,nml_ins,color_key_inc
0,20001,41692,5,54353.0,96.0,11.0,32.0,0.782609,0.383988,0.960,#fed884
1,20002,52867,6,49452.0,95.4,8.0,58.0,0.739130,0.336126,0.954,#fdc171
2,20003,26330,5,63600.0,97.6,8.0,49.0,0.782609,0.474292,0.976,#fff7b2
3,20004,1610,4,117432.0,98.7,5.0,14.0,0.826087,1.000000,0.987,#006837
4,20005,12311,2,76658.0,95.5,6.0,20.0,0.913043,0.601813,0.955,#d7ee8a


In [23]:
colors = []

for pop in econ_select3['nml_emp'].tolist():
    try:
        color = calculate_color(pop)
    except KeyError:
        color = (0, 0, 0, 0.3)
        print('key error')
    colors.append(color)

In [24]:
econ_select3['color_key_emp'] = colors

econ_select3.head()

,zip,Total Population,Unemployment Rate,Per Capita Income,Percent Insured,hospital facilities,doctors,nml_emp,nml_inc,nml_ins,color_key_inc,color_key_emp
0,20001,41692,5,54353.0,96.0,11.0,32.0,0.782609,0.383988,0.960,#fed884,#70c164
1,20002,52867,6,49452.0,95.4,8.0,58.0,0.739130,0.336126,0.954,#fdc171,#8ccd67
2,20003,26330,5,63600.0,97.6,8.0,49.0,0.782609,0.474292,0.976,#fff7b2,#70c164
3,20004,1610,4,117432.0,98.7,5.0,14.0,0.826087,1.000000,0.987,#006837,#51b35e
4,20005,12311,2,76658.0,95.5,6.0,20.0,0.913043,0.601813,0.955,#d7ee8a,#16914d


In [25]:
colors = []

for pop in econ_select3['nml_ins'].tolist():
    try:
        color = calculate_color(pop)
    except KeyError:
        color = (0, 0, 0, 0.3)
        print('key error')
    colors.append(color)

In [26]:
econ_select3['color_key_ins'] = colors

econ_select3.head()

,zip,Total Population,Unemployment Rate,Per Capita Income,Percent Insured,hospital facilities,doctors,nml_emp,nml_inc,nml_ins,color_key_inc,color_key_emp,color_key_ins
0,20001,41692,5,54353.0,96.0,11.0,32.0,0.782609,0.383988,0.960,#fed884,#70c164,#0a7b41
1,20002,52867,6,49452.0,95.4,8.0,58.0,0.739130,0.336126,0.954,#fdc171,#8ccd67,#0b7d42
2,20003,26330,5,63600.0,97.6,8.0,49.0,0.782609,0.474292,0.976,#fff7b2,#70c164,#06733d
3,20004,1610,4,117432.0,98.7,5.0,14.0,0.826087,1.000000,0.987,#006837,#51b35e,#036e3a
4,20005,12311,2,76658.0,95.5,6.0,20.0,0.913043,0.601813,0.955,#d7ee8a,#16914d,#0b7d42


In [27]:
zip_data.head()

,zip,city,county,median_household_income,population,feature,area,density,nml_dens,nml_inc,color_key
0,20001,"Washington, DC",District Of Columbia,100447.0,41692,"{'type': 'Feature', 'geometry': {'type': 'Poly...",5.835524,7144.516970,0.759751,0.386274,#0062ce
1,20002,"Washington, DC",District Of Columbia,82022.0,52867,"{'type': 'Feature', 'geometry': {'type': 'Poly...",14.228399,3715.597280,0.964587,0.284281,#0048db
2,20003,"Washington, DC",District Of Columbia,118607.0,26330,"{'type': 'Feature', 'geometry': {'type': 'Poly...",6.447784,4083.573672,0.478169,0.486800,#007cc1
3,20004,"Washington, DC",District Of Columbia,144583.0,1610,"{'type': 'Feature', 'geometry': {'type': 'Mult...",0.904905,1779.192084,0.025057,0.630593,#00a1ae
4,20005,"Washington, DC",District Of Columbia,94506.0,12311,"{'type': 'Feature', 'geometry': {'type': 'Poly...",1.122787,10964.678013,0.221204,0.353387,#005ad2


In [28]:
econ_main = zip_data[['zip','feature','area','density']].rename(index=str, columns={'area': 'area km'})

econ_main = econ_main.merge(econ_select3, on='zip', how='left').fillna(0)

In [29]:
econ_main.to_csv('../primary_data.csv', encoding='utf-8')

In [30]:
feat_list = []
for feature in econ_main['feature']:
    feat_list.append(ast.literal_eval(feature))

geoj_clean = {'type': 'FeatureCollection', 'features': feat_list}

with open('../zcta_map/resources/features.json', 'w') as outfile:  
    json.dump(geoj_clean, outfile)